In [24]:
%load_ext autoreload
%autoreload 2

import os

import requests

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
container_uri = "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.7.1-cpu-py3"
container_ecr_root = container_uri.partition("/")[0]

models_uri = "s3://sagemaker-us-east-1-761655029435/sagemaker/DEMO-pytorch-rea/mme-artifactsv2/"

env = {
    "SAGEMAKER_CONTAINER_LOG_LEVEL": 20,
    "SAGEMAKER_PROGRAM": "inference.py",
    "SAGEMAKER_REGION": "us-east-1",
    "SAGEMAKER_SUBMIT_DIRECTORY": "s3://sagemaker-us-east-1-761655029435/pytorch-inference-2021-04-29-15-43-05-877/model.tar.gz",
    "SAGEMAKER_MULTI_MODEL": "true",
}

In [11]:
!aws ecr get-login-password | docker login --username AWS --password-stdin $container_ecr_root

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [12]:
!docker pull $container_uri

1.7.1-cpu-py3: Pulling from pytorch-inference

a5e7af40: Pulling fs layer 
39a868b3: Pulling fs layer 
b10cca85: Pulling fs layer 
bdcfbe9a: Pulling fs layer 
b3724899: Pulling fs layer 
66df3ed3: Pulling fs layer 
b110b773: Pulling fs layer 
b4a5ad10: Pulling fs layer 
ba4a455f: Pulling fs layer 
fb732584: Pulling fs layer 
0f518039: Pulling fs layer 
8ab2574f: Pulling fs layer 
f57e7177: Pulling fs layer 
56f82a8c: Pulling fs layer 
c2709796: Pulling fs layer 
d36ff2e5: Pulling fs layer 
584cc727: Pulling fs layer 
dcd505e4: Pulling fs layer 
45c7c905: Pulling fs layer 
c6b2ff52: Pulling fs layer 
52e406f6: Pulling fs layer 
Digest: sha256:8bfeebf71c7af9e0146d45d61539657fc413e8eb740bbc2ed52c9e7facc141c92K
Status: Downloaded newer image for 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.7.1-cpu-py3
763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.7.1-cpu-py3


In [43]:
with open("env.txt", "w") as f:
    for k, v in env.items():
        f.write(f"{k}={v}\n")

In [35]:
env_opts = " ".join(map(lambda kv: f'-e "{kv[0]}={kv[1]}"', env.items()))
print(env_opts)

-e "SAGEMAKER_CONTAINER_LOG_LEVEL=20" -e "SAGEMAKER_PROGRAM=inference.py" -e "SAGEMAKER_REGION=us-east-1" -e "SAGEMAKER_SUBMIT_DIRECTORY=s3://sagemaker-us-east-1-761655029435/pytorch-inference-2021-04-29-15-43-05-877/model.tar.gz" -e "SAGEMAKER_MULTI_MODEL=true"


In [36]:
#run_cmd = f"docker run -it -p 8080:8110 -p 8081:8111 {env_opts} {container_uri}"
run_cmd = f"docker run -it -p 8110:8080 -p 8111:8081 {env_opts} {container_uri}"
print(run_cmd)

docker run -it -p 8110:8080 -p 8111:8081 -e "SAGEMAKER_CONTAINER_LOG_LEVEL=20" -e "SAGEMAKER_PROGRAM=inference.py" -e "SAGEMAKER_REGION=us-east-1" -e "SAGEMAKER_SUBMIT_DIRECTORY=s3://sagemaker-us-east-1-761655029435/pytorch-inference-2021-04-29-15-43-05-877/model.tar.gz" -e "SAGEMAKER_MULTI_MODEL=true" 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.7.1-cpu-py3


In [44]:
#run_cmd = f"docker run -it -p 8080:8110 -p 8081:8111 {env_opts} {container_uri}"
run_cmd = f"docker run --env-file env.txt -it -p 8110:8080 -p 8111:8081 {container_uri}"
print(run_cmd)

docker run --env-file env.txt -it -p 8110:8080 -p 8111:8081 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.7.1-cpu-py3


^ Run the above in a terminal window

In [34]:
ping_resp = requests.get("http://0.0.0.0:8110/ping")
print(ping_resp)
ping_resp.json()

<Response [200]>


{'status': 'Healthy'}

In [37]:
list_resp = requests.get(
    "http://0.0.0.0:8111/models",
)
print(list_resp)
list_resp.json()

<Response [200]>


{'models': []}

In [54]:
load_resp = requests.post(
    "http://0.0.0.0:8111/models",
    json={
        "model_name": "Model1.tar.gz",
        "url": f"{models_uri}Model1.tar.gz",
#         "url": "/opt/ml/models/Model1.tar.gz",
    },
)
print(load_resp)
load_resp.json()

<Response [404]>


{'code': 404,
 'type': 'ModelNotFoundException',
 'message': 'Model not found at: s3://sagemaker-us-east-1-761655029435/sagemaker/DEMO-pytorch-rea/mme-artifactsv2/Model1.tar.gz'}

In [41]:
!aws s3 ls s3://sagemaker-us-east-1-761655029435/sagemaker/DEMO-pytorch-rea/mme-artifactsv2/

2021-05-04 01:25:54    2353140 Model1.tar.gz
2021-05-04 01:25:54    2353140 Model2.tar.gz


TODO: Why doesn't the container seem to see 'SAGEMAKER_MULTI_MODEL'? Note 'Model Store: /home/model-server' in output, which contradicts https://github.com/aws/sagemaker-pytorch-inference-toolkit/blob/6610a410c0cf40bcf15267abe722d20d50e77bcf/src/sagemaker_pytorch_serving_container/torchserve.py#L49

In [50]:

ENTRYPOINT = ["python", "/usr/local/bin/dockerd-entrypoint.py"]
CMD = ["torchserve", "--start", "--ts-config", "/home/model-server/config.properties", "--model-store", "/home/model-server/"]

In [51]:
" ".join(CMD)

'torchserve --start --ts-config /home/model-server/config.properties --model-store /home/model-server/'